# Using InlineAgent Runnable with Return of Control (RoC)

The following examples will show how you can begin integrating with the InlineAgent Runnable.

### Prerequisites:
1. Set your aws credentials for your environment, example: https://docs.aws.amazon.com/cli/v1/userguide/cli-configure-envvars.html#envvars-set.
1. Ensure that langchain, langgraph are installed in the environment and that the local langchain-aws is accessible from the path or installed into the environment. 

## Example 1: Create a mortgage agent that determines the interest rate
In this example, we create a mortgage agent with two tools. The first tool will return the asset values of a given asset holder. The second tool will return the interest rate for a given asset holder with a given asset value.

### Step 1: Define the tools for the agent

In [1]:
from langchain_core.tools import tool
from langchain_aws.agents import BedrockInlineAgentsRunnable
from langchain_core.messages import HumanMessage

@tool("AssetDetail::getAssetValue")
def get_asset_value(asset_holder_id: str) -> str:
    """
    Get the asset value for an owner id

    Args:
        asset_holder_id: The asset holder id

    Returns:
        The asset value for the given asset holder
    """
    return f"The total asset value for {asset_holder_id} is 100K"

@tool("AssetDetail::getMortgageRate")
def get_mortgage_rate(asset_holder_id: str, asset_value: str) -> str:
    """
    Get the mortgage rate based on asset value

    Args:
        asset_holder_id: The asset holder id
        asset_value: The value of the asset

    Returns:
        The interest rate for the asset holder and asset value
    """
    return f"The mortgage rate for {asset_holder_id} with asset value of {asset_value} is 8.87%"

tools = [get_asset_value, get_mortgage_rate]
print(tools)

[StructuredTool(name='AssetDetail::getAssetValue', description='Get the asset value for an owner id\n\nArgs:\n    asset_holder_id: The asset holder id\n\nReturns:\n    The asset value for the given asset holder', args_schema=<class 'langchain_core.utils.pydantic.AssetDetail::getAssetValue'>, func=<function get_asset_value at 0x1071cf100>), StructuredTool(name='AssetDetail::getMortgageRate', description='Get the mortgage rate based on asset value\n\nArgs:\n    asset_holder_id: The asset holder id\n    asset_value: The value of the asset\n\nReturns:\n    The interest rate for the asset holder and asset value', args_schema=<class 'langchain_core.utils.pydantic.AssetDetail::getMortgageRate'>, func=<function get_mortgage_rate at 0x1071cf880>)]


### Step 2: Define the agent configuration and create the runnable

In [2]:
foundation_model = 'anthropic.claude-3-sonnet-20240229-v1:0'
instructions = "You are a friendly and cheerful assistant that answers questions valiantly."
inline_agent_config = {
    "foundation_model": foundation_model,
    "instruction": instructions,
    "tools": tools,
    "enable_trace": False
}

runnable = BedrockInlineAgentsRunnable.create(
    region_name="us-west-2",
    inline_agent_config=inline_agent_config
)

print("BedrockInlineAgentsRunnable created successfully.")
messages = [HumanMessage(content="what is my mortgage rate for id AVC-1234")]


BedrockInlineAgentsRunnable created successfully.


### Step 3: Invoke the agent

In [3]:
# Invoke the inlineAgent
output = runnable.invoke(messages)
print(output)

content='' additional_kwargs={'session_id': '91f079b5-6b4f-4584-84d9-bf02c85802ec', 'trace_log': '[]', 'roc_log': '{"returnControl": {"invocationId": "dbd115dd-3bc0-4816-b848-ee56e78409c7", "invocationInputs": [{"functionInvocationInput": {"actionGroup": "AssetDetail", "actionInvocationType": "RESULT", "agentId": "INLINE_AGENT", "function": "getAssetValue", "parameters": [{"name": "asset_holder_id", "type": "string", "value": "AVC-1234"}]}}]}}'} response_metadata={} tool_calls=[{'name': 'AssetDetail::getAssetValue', 'args': {'asset_holder_id': 'AVC-1234'}, 'id': '888727aa-f739-45df-9559-12d094832243', 'type': 'tool_call'}]


## Example2: Updating the configs on the fly
* You can modify any agent configuration for each request by providing the updates in the `inline_agent_configuration`. 
* You can also start a new conversation by providing a `session_id`

**Note:** Updates are additive and will build on previous configurations, if you want to remove a config, pass in a NoneType


In [4]:
inline_agent_config = {
    "instruction": "You are now a pirate that answers questions in Haiku! You MUST end all sentences in 'Yarr!'",
    "tools": None,
    "enable_trace": True
}
output = runnable.invoke([HumanMessage(content="What are cars?")], inline_agent_config=inline_agent_config, session_id="test-sesh-1")
print(output)

content="Metal horses tamed, yarr\nEngines thrumming, wheels a-spin\nHumans' chariots, yarr" additional_kwargs={'session_id': 'test-sesh-1', 'trace_log': '[{"sessionId": "test-sesh-1", "trace": {"orchestrationTrace": {"modelInvocationInput": {"text": "{\\"system\\":\\" You are now a pirate that answers questions in Haiku! You MUST end all sentences in \'Yarr!\' You have been provided with a set of functions to answer the user\'s question. You must call the functions in the format below: <function_calls>   <invoke>     <tool_name>$TOOL_NAME</tool_name>     <parameters>       <$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>       ...     </parameters>   </invoke> </function_calls> Here are the functions available: <functions>    </functions>  You will ALWAYS follow the below guidelines when you are answering a question: <guidelines> - Think through the user\'s question, extract all data from the question and the previous conversations before creating a plan. - Never assume any paramet

## Example3: Using CodeInterpretter functionality

The code interpretation(CI) enables your agent to generate, run, and troubleshoot your application code in a secure test environment. You can enable CI on an Inline Agent by passing in enable_code_interpreter = True while creating the runnable or making the invoke request.

In [5]:
from langchain_core.tools import tool
from langchain_aws.agents import BedrockInlineAgentsRunnable
from langchain_core.messages import HumanMessage
foundation_model = 'anthropic.claude-3-sonnet-20240229-v1:0'
instructions = "You are an agent who helps with getting the mortgage rate based on the current asset valuation"
inline_agent_config = {
    "foundation_model": foundation_model,
    "instruction": instructions,
    "enable_code_interpreter": True,
    "enable_trace": False
}

runnable = BedrockInlineAgentsRunnable.create(
    region_name="us-west-2",
    inline_agent_config=inline_agent_config
)

print("BedrockInlineAgentsRunnable and AgentExecutor created successfully.")

# Invoke the agent.
output = runnable.invoke(
    [HumanMessage(content="Can you please help plot a line graph for the following values - [1,199,1999, 199999, 122] against an increasing x value [1,2,3,4,5]?")]
)

print("Agent Output:")
print(output)

BedrockInlineAgentsRunnable and AgentExecutor created successfully.
Agent Output:
content='I have plotted a line graph with the given y-values [1, 199, 1999, 199999, 122] against the increasing x-values [1, 2, 3, 4, 5]. The line graph has been saved as an image file called line_graph.png.' additional_kwargs={'session_id': '8b2d281b-ae18-4794-a203-f5f922e86d1e', 'trace_log': '[]', 'files': [{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\x80\x00\x00\x01\xe0\x08\x06\x00\x00\x005\xd1\xdc\xe4\x00\x00\x009tEXtSoftware\x00Matplotlib version3.9.0, https://matplotlib.org/4\x04H\xb6\x00\x00\x00\tpHYs\x00\x00\x0fa\x00\x00\x0fa\x01\xa8?\xa7i\x00\x00i+IDATx\x9c\xed\xddy|\x13u\xfe?\xf0W\xd26I\xaf\xb4\x85\xdePJ\x01)PJQ\xd0nQ.\xa9\x14\xa8\xac\xac\xee\xaax!\xa2"\x0b(\xb0\n\xb2\x8bx\xae\xb0 \n.(\xae.\xc7\xba\xfcV`\xbf\xe2Q\xb0l)\x97BE(\x14\xda\x02\xe5*\x14\xa1\xe9\x014\xe9}$\x9f\xdf\x1fm\x86\x86\x16z\xd0f\x92\xe6\xf5|<\xe6\xb1d\xe6=\x93\xf7$+y3\xef\xf9\xccG!\x84\x10 """"\x87\xa1\x94;\x01"""